In [1]:
import json

In [2]:
data = json.load(open('./result_json/14.03.19-14:16.json'))

In [3]:
data[0]

{'post_id': 155,
 'owner_id': 1125938,
 'dialog_id': '140319_0',
 'post_text': 'не упади со скалы!',
 'post_num_comments': 0,
 'persona_id': 1595309,
 'other_id': 421994787,
 'dialog': [{'persona_type': 'other', 'num_answers': 0, 'text': '🙀 👀\n'},
  {'persona_type': 'persona',
   'num_answers': 0,
   'text': '[id1424187|Алина], я тут на эти фотки наткнулась и не смогла удержаться.'},
  {'persona_type': 'persona',
   'num_answers': 0,
   'text': '[id1424187|Алина], вообще-то я думала Вы меня прибьете.😜'}]}

In [4]:
import pandas as pd

In [5]:
    FILE_NAME = "14.03.19-14:16"
    source_comments = pd.read_csv(f'./wall/girl_wall/comments/{FILE_NAME}.csv', index_col=0)
    source_answers = pd.read_csv(f'./wall/girl_wall/answers/{FILE_NAME}.csv', index_col=0)
    source_posts = pd.read_csv(f'./wall/girl_wall/posts/{FILE_NAME}.csv', index_col=0)
    source_answers = source_answers.loc[:, ~source_answers.columns.str.contains('^Unnamed')]
    source_comments = source_comments.loc[:, ~source_comments.columns.str.contains('^Unnamed')]
    source_posts = source_posts.loc[:, ~source_posts.columns.str.contains('^Unnamed')]
    source_comments.replace({pd.np.NaN: None}, inplace=True)
    source_answers.replace({pd.np.NaN: None}, inplace=True)
    source_posts.replace({pd.np.NaN: None}, inplace=True)
    source_comments.drop_duplicates(inplace=True)
    source_answers.drop_duplicates(inplace=True)
    source_posts.drop_duplicates(inplace=True)
    source_comments.sort_values(by=['post_id', 'owner_id', 'date'], inplace=True)
    source_answers.sort_values(by=['post_id', 'owner_id', 'date'], inplace=True)

In [6]:
import scripts.wall_to_json as t

In [7]:
result = t.get_dialogs_from_answers(source_answers, source_comments, None)

In [8]:
with open('./result.txt', 'w') as f:
    for num, dialog in enumerate(result):
        f.write(str(num)+" "+'\n')
        for _, row in pd.concat(dialog).iterrows():
            f.write(str(row['text'])+'\n')
        f.write('\n')

In [16]:
pd.concat(result[179])

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
81048,1298,1549222585,92490,7,5040411,[],1297,None,None,А куда дальше передаётся Тетра-пак? Кто его пе...
2546,1299,1549224023,5040411,0,5040411,[1298],1297,None,None,"[id92490|Виктория], Я всего лишь волонтёр. Дум..."
2547,1300,1549224104,92490,0,5040411,[1298],1297,None,None,"Спасибо за ответ! Собираю информацию, так как ..."
2548,1301,1549224191,5040411,0,5040411,[1298],1297,None,None,"[id92490|Виктория], я думаю тут ошибка. Перера..."
2549,1302,1549224450,5040411,0,5040411,[1298],1297,1300,92490,"[id92490|Виктория],"
2550,1303,1549224464,92490,0,5040411,[1298],1297,1301,5.04041e+06,"[id5040411|Полина], если бы! Это не просто ком..."
2551,1304,1549224778,5040411,0,5040411,[1298],1297,1303,92490,"[id92490|Виктория], дело в том, что для беспр..."
2552,1305,1549225223,92490,0,5040411,[1298],1297,None,None,Спасибо:) буду ждать!


In [106]:
import time
time.ctime(1542727975)

'Tue Nov 20 18:32:55 2018'

In [28]:
result[0]

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
24608,245,1542727975,1424187,2,1595309,[],244,None,None,Спасибо!!! Даже не верится... так давно это было)
310,246,1542728259,1595309,0,1595309,[245],244,None,None,"[id1424187|Алина], я тут на эти фотки наткнула..."
0,247,1542728358,1595309,0,1595309,[245],244,None,None,"[id1424187|Алина], вообще-то я думала Вы меня ..."


In [29]:
result = list(map(pd.concat, result))

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [86]:
def remove_unnecessary_info(table):
    table = table.reset_index(drop=True)
    table = table[['comment_id', 'date', 'from_id',\
                   'num_answer', 'owner_id', 'parents_stack',\
                   'post_id', 'text']]
    return table

result_dialogs = []
acc = [result[0]]
for dialog in result[1:]:
    tail = acc[-1].tail(1)
    tail = remove_unnecessary_info(tail)
    head = dialog.head(1)
    head = remove_unnecessary_info(head)
    if (tail == head).all(axis=1).item():
        acc.append(dialog)
    else:
        completed_dialog = pd.concat(acc)
        completed_dialog = completed_dialog.drop_duplicates()
        result_dialogs.append(completed_dialog)
        acc = [dialog]

In [87]:
result_dialogs[23]

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
0,606,1549643234,155880665,3,2244699,[],605,None,None,"Поздравляю Юру 🎊🎊🎊🎊такой красивый пес🐶, пережи..."
0,608,1549646128,21753144,0,2244699,[606],605,None,None,Вот это новости!!!!! Супер!!! Очень рада Натал...
0,609,1549646209,2244699,0,2244699,[606],605,608,2.17531e+07,"[id21753144|Юлия], спасибо)) 🤗🥰 А я то как рад..."


In [88]:
set(map(lambda x: len(set(x['from_id'])), result_dialogs))

{1, 2, 3, 4, 9}

In [98]:
a = list(filter(lambda x: len(set(x['from_id'])) == 4, result_dialogs))

In [102]:
[i['text'] for _, i in a[0].iterrows()]

['[id205007224|Игорь], Есть он и мне очень приятно такое к себе отношение,жаль я не знаю кто это) \nНу ты,считать можешь что угодно)))',
 '[id4491403|Юлия], может бывший твой заскучал , не думала об этом ?',
 '[id225891086|Алексей], Добрый человек прислал, возможно, флаг ему в руки для изгнания тёмных сил.',
 '[id296145973|Александр], А ты джедай? Все время темные силы гоняешь .😁😁😁',
 '[id205007224|Игорь], Можно и так сказать, "гонитель тёмных сил", до "истребителя" ещё расти.']

In [103]:
a[0]

,comment_id,date,from_id,num_answer,owner_id,parents_stack,post_id,reply_to_comment,reply_to_user,text
0,59571,1540844664,4491403,0,4491403,[59563],59495,59568,2.05007e+08,"[id205007224|Игорь], Есть он и мне очень прият..."
0,59576,1540857176,225891086,0,4491403,[59563],59495,59571,4.4914e+06,"[id4491403|Юлия], может бывший твой заскучал ,..."
0,59577,1540874859,296145973,0,4491403,[59563],59495,59576,2.25891e+08,"[id225891086|Алексей], Добрый человек прислал,..."
1839,59583,1540915520,205007224,0,4491403,[59563],59495,59577,2.96146e+08,"[id296145973|Александр], А ты джедай? Все врем..."
0,59586,1540916201,296145973,0,4491403,[59563],59495,59583,2.05007e+08,"[id205007224|Игорь], Можно и так сказать, ""гон..."
